In [5]:
!pip install cirq --quiet


In [2]:
#@title Bell Inequality Simulator
## Imports
import numpy as np
import cirq

## Utility functions
def bitstring(bits):
    return ''.join('1' if e else '_' for e in bits)


class CHSHBell(object):

    def __init__(self, repetitions=1):
        super().__init__()
        self.repetitions = repetitions
    
    def setup(self):
        alice_recv = cirq.NamedQubit("x")
        bob_recv = cirq.NamedQubit("y")
        alice_retn = cirq.NamedQubit("a")
        bob_retn = cirq.NamedQubit("b")
        return alice_recv, bob_recv, alice_retn, bob_retn

    def make_quantum_bell_test_circuit(self, alice_recv, bob_recv, alice_retn, bob_retn):
        circuit = cirq.Circuit()

        # Prepare shared entangled state.
        circuit.append([cirq.H(alice_retn), cirq.CNOT(alice_retn, bob_retn)])

        # Received bits are randomized.
        circuit.append([cirq.H(alice_recv), cirq.H(bob_recv)])

        # Players do a sqrt(X) based on their received bit.
        circuit.append(
            [cirq.X(alice_retn) ** -0.25,
             cirq.CNOT(alice_recv, alice_retn) ** 0.5,
             cirq.CNOT(bob_recv, bob_retn) ** 0.5]
        )
        # Then results are recorded.
        circuit.append(
            [
                cirq.measure(alice_recv, key='x'),
                cirq.measure(alice_retn, key='a'),
                cirq.measure(bob_recv, key='y'),
                cirq.measure(bob_retn, key='b'),
            ]
        )
        return circuit

    def run_quantum_bell_test(self):
        alice_recv, bob_recv, alice_retn, bob_retn = self.setup()
        self.circuit = self.make_quantum_bell_test_circuit(alice_recv, bob_recv, alice_retn, bob_retn)
        print('Quantum Best Strategy Circuit:')
        print(self.circuit)

        print(f'Simulating {self.repetitions} repetitions...')
        result = cirq.Simulator().run(program=self.circuit, repetitions=self.repetitions)

        # Collect results.
        a = np.array(result.measurements['a'][:, 0])
        b = np.array(result.measurements['b'][:, 0])
        x = np.array(result.measurements['x'][:, 0])
        y = np.array(result.measurements['y'][:, 0])
        outcomes = a ^ b == x & y
        win_percent = len([e for e in outcomes if e]) * 100 / self.repetitions

        # Print data.
        print('Results')
        print('a:', bitstring(a))
        print('b:', bitstring(b))
        print('x:', bitstring(x))
        print('y:', bitstring(y))
        print('(a XOR b) == (x AND y):\n  ', bitstring(outcomes))
        print(f'Win rate: {win_percent}%')

    def make_classical_bell_test_circuit(self, alice_recv, bob_recv, alice_retn, bob_retn):
        circuit = cirq.Circuit()
        # Received bits are randomized.
        circuit.append([cirq.H(alice_recv), cirq.H(bob_recv)])
        # Alice and Bob simply return 0s.
        # Then results are recorded.
        circuit.append(
            [
                cirq.measure(alice_recv, key='x'),
                cirq.measure(alice_retn, key='a'),
                cirq.measure(bob_recv, key='y'),
                cirq.measure(bob_retn, key='b'),
            ]
        )
        return circuit

    def run_classical_bell_test(self):
        alice_recv, bob_recv, alice_retn, bob_retn = self.setup()
        self.circuit = self.make_classical_bell_test_circuit(alice_recv, bob_recv, alice_retn, bob_retn)
        print('Classical Best Strategy Circuit:')
        print(self.circuit)

        print(f'Simulating {self.repetitions} repetitions...')
        result = cirq.Simulator().run(program=self.circuit, repetitions=self.repetitions)

        # Collect results.
        a = np.array(result.measurements['a'][:, 0])
        b = np.array(result.measurements['b'][:, 0])
        x = np.array(result.measurements['x'][:, 0])
        y = np.array(result.measurements['y'][:, 0])
        outcomes = a ^ b == x & y
        win_percent = len([e for e in outcomes if e]) * 100 / self.repetitions

        # Print data.
        print('Results')
        print('a:', bitstring(a))
        print('b:', bitstring(b))
        print('x:', bitstring(x))
        print('y:', bitstring(y))
        print('(a XOR b) == (x AND y):\n  ', bitstring(outcomes))
        print(f'Win rate: {win_percent}%')


In [3]:
belltest = CHSHBell(repetitions=100)
belltest.run_quantum_bell_test()

Quantum Best Strategy Circuit:
                        ┌──────┐
a: ───H───@───X^-0.25────X^0.5─────M───
          │              │
b: ───────X───X^0.5──────┼────M────────
              │          │
x: ───H───────┼──────────@─────────M───
              │
y: ───H───────@──────────M─────────────
                        └──────┘
Simulating 100 repetitions...
Results
a: __1_1__11_1_11__11__1_11___1111___1_1___11_1__1_1__1__1111_1_1_1__111111____1_111___1_1___1111111___
b: _11_1__11______1_1_1__11___11111__111_1_1_11_1__1__1111___1111_1___11111_111__1____11_1___1_1___1__1
x: 11___1_1_11111_____11__________1__111_1_11__11111_111111111__1_11_11___111_11_1_1_1111___1_1_111__1_
y: 11_111____1__111___111____11_11111_11111_11___1__1__11_1111___1__11_111_111111_11__11_1_1_111111_1_1
(a XOR b) == (x AND y):
   _1111_111111_11__1111111111111111111_11111_11_11111111111111_11111111111_1_1111_1111_11111111111111_
Win rate: 86.0%


In [4]:
belltest.run_classical_bell_test()

Classical Best Strategy Circuit:
a: ───M───────

b: ───M───────

x: ───H───M───

y: ───H───M───
Simulating 100 repetitions...
Results
a: ____________________________________________________________________________________________________
b: ____________________________________________________________________________________________________
x: _11______111__1_111_11___1_11111_11_1______1__1_11_11__1_11111__11111__111__11_1_1_____1_1_1_1_1___1
y: 11__1_1111_1____1_111_11111_11_11_111__1_1__1_1_1_1_1_1___11111__111111_1_111_1__11__1_1__1____11_11
(a XOR b) == (x AND y):
   1_1111111_1_1111_1_1_1111_11__1_11_1_111111111_1_111_11111____111____111_111_1111_11111_1111111_111_
Win rate: 71.0%
